In [1]:
import numpy as np #Vectorization calculations
import matplotlib.pyplot as plt #Plotting cost function
import os #Find proper path in file directory

#Read images files
from PIL import Image #Processing images as RGB

#Test Neural Networks with randoms hyperparameters
#from random import randint


In [2]:
"""
Load images in a folder as a list RGB matrices
"""
def load_images(folder):
    image_size = 64
    images = []
    counter =0
    for filename in os.scandir(folder):
            counter += 1
            
            #Consider edge-case where there is a folder inside the list of images
            if(os.path.isdir(filename)):
                continue
            
            #Path/to/file + /filename.extension
            img = Image.open(os.path.join(folder,os.path.basename(filename)))
        
            #standardize image size for easier computation
            img = img.resize((image_size, image_size))
            img = np.array(img.convert(mode = "RGB"))
                                    
            if(counter %100 == 0):
                print("Iteration: ", counter,": ",filename)
            
            images.append(img)
            
    return np.asarray(images)




In [3]:
"""
Load & preprocess images from two file paths (normalPath, pneumoniaPath)
"""
def load_xy_set(pathOne, pathTwo):
    #First, process normal x-ray images
    #Add normal images and place them as zeros
    x = load_images(pathOne)
    y = np.zeros((1,x.shape[0]))
    
    x1 = load_images(pathTwo)
    
    #Add pneumonia images and mark them as ones
    x = np.concatenate((x, x1),axis = 0)
    y = np.concatenate((y, np.ones((1,x.shape[0]-y.shape[1]))), axis= 1)

    
    assert (y.shape[1] == x.shape[0])
    
    #Reshape X for easier computation
    m  =x.shape[0] #Number of training examples

    x = x.reshape(m,-1).T/255 #(Width*height*layers, training_examples)
    
    return x, y


In [4]:
"""
Created a seperate, much smaller training_set to test pre-processing functions
"""

#TEST FILES (5 images per file)

pathOne = "chest_xray/trainTEMP/normal"
pathTwo = "chest_xray/trainTEMP/pneumonia"
x_temp, y_temp = load_xy_set(pathOne, pathTwo)

#Verify contents and shapes match each other
#x_train.shape ==> (total_training_examples,width,height,RGB)
print("x_final: ",x_temp.shape)
#print("x_final[0]: ", x_temp[:,0])

#y_train.shape ==> (1,total_training_examples)
print("y_final: ",y_temp.shape)
#print("y_final[0]:", y_temp[:,0])

x_final:  (12288, 10)
y_final:  (1, 10)


In [5]:
"""
Helper activation functions: 
def sigmoid() #Binary classification; used in output layer
def relu() #Used in hidden layers
def tanh() #Used in hidden layers
""" 
def sigmoid(Z):
    #Treat Z as np.array
    #Use for last layer activation function
    s = 1/(1+np.exp(-Z))
    
    return s
   
    
def relu(Z):
    #Treat Z as np.array
    s = (abs(Z)+Z)/2
    return s

#tanh() function can be used with numpy:
#tanh() --> np.tanh()
    


In [6]:
"""
Verify that relu & sigmoid & tanh work
"""
Z = np.array(([-1,0,2], [-10,0,1],[1,2,10]))
#print(Z.shape)
print("Sigmoid: \n",sigmoid(Z))
print("Relu: \n",relu(Z))
print("Tanh: \n",np.tanh(Z))

Sigmoid: 
 [[2.68941421e-01 5.00000000e-01 8.80797078e-01]
 [4.53978687e-05 5.00000000e-01 7.31058579e-01]
 [7.31058579e-01 8.80797078e-01 9.99954602e-01]]
Relu: 
 [[ 0.  0.  2.]
 [ 0.  0.  1.]
 [ 1.  2. 10.]]
Tanh: 
 [[-0.76159416  0.          0.96402758]
 [-1.          0.          0.76159416]
 [ 0.76159416  0.96402758  1.        ]]


In [7]:
"""
initialize_weights & biases
Get the weights & biases for each individual layer & respective neurons/nodes in L-layer NN
"""

def initialize_parameters(layer_dims):
    parameters = {}
    L = len(layer_dims) #Number of layers in the network
    
    for l in range(1,L):
        #print all of the dimensions that weight & bias would have in layer l
        #print("Layer_dimensions: ",layer_dims[l], layer_dims[l-1])
        parameters['W'+str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1])*.01
        parameters['b'+str(l)] = np.random.randn(layer_dims[l],1)
        
        assert(parameters['W'+str(l)].shape == (layer_dims[l],layer_dims[l-1]))
        assert(parameters['b'+str(l)].shape == (layer_dims[l], 1))
        
        
    return parameters

In [8]:
"""
Test the initialize_parameters function:
-Confirm that it produces the proper shapes for a given set of layer_dims

Pseudo-code:

for x in range(1,len(layer_dims)):
 weight.shape = (layer_dims[x], [x-1])
 bias.shape = (layer_dims[x], [x-1])
 
"""
layer_dims = [x_temp.shape[0],5, 4, 3, 1]
counter = 1
print("X_input: ", x_temp.shape[0])

parameters = initialize_parameters(layer_dims)

#Test for proper bias & weight shapes
for x in parameters:
    print(x, ": ", parameters[x].shape)

X_input:  12288
W1 :  (5, 12288)
b1 :  (5, 1)
W2 :  (4, 5)
b2 :  (4, 1)
W3 :  (3, 4)
b3 :  (3, 1)
W4 :  (1, 3)
b4 :  (1, 1)


In [9]:
"""
#forward propogation
def forward_propogate()
Calculate Z for a single layer

Final Z dimension should be (W[l-1].shape[0],1)
"""

def linear_forward(A, W, b):
    
    #Calculate Z in Neuron
    Z = np.dot(W, A) + b


    assert(Z.shape == (W.shape[0], A.shape[1]))
    
    #Save cache for backward propogation    
    cache = (A, W, b)
    
    return Z, cache






In [10]:
"""
#forward propogation
calculate the activation function portion of forward propogation

Use either (Relu and sigmoid) or (tanh and sigmoid) for best performance
"""


def activation_forward(A_prev, W, b, activation):
    
    if activation == "sigmoid":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A = sigmoid(Z)
        
    elif activation == "relu":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A = relu(Z)
        
    elif activation == "tanh":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A = np.tanh(Z)
        
    assert(A.shape == (W.shape[0], A_prev.shape[1]))
    cache = (linear_cache, Z)
    return A, cache

In [11]:
"""
#Forward propogation
put together the previous forward_propogation helper functions into a single model
"""

def forward_propogation(X, parameters, hidden_activation):
    caches = []
    A = X
    L = len(parameters)//2
       
    for l in range(1, L):
        A_prev = A
        A, cache = activation_forward(A_prev, parameters["W"+str(l)], parameters["b"+str(l)], hidden_activation)
        caches.append(cache)
    
    #Compute final prediction
    AL, cache = activation_forward(A, parameters["W"+str(L)], parameters["b"+str(L)], "sigmoid")
    caches.append(cache)
    assert(AL.shape == (1,X.shape[1]))
    
    return AL, caches

In [12]:
"""
#Cost function
def compute_cost()

Implement loss function for binary classifcation:
-1/m sum(Ylog(A) + (1-Y)log(1-A))
"""

def compute_cost(A_final, Y):
    
    m = Y.shape[1]
    
    
    cost = -1/m * np.sum(Y*np.log(A_final) + (1-Y)*np.log(1-A_final))
    
    cost = np.squeeze(cost)
    assert(cost.shape == ())
    
    return cost


In [13]:
"""
#Backward propogation
def  linear_backward()
calculate gradients (derivatives) of a layer,i, 
"""

def linear_backward(dZ, cache):
    
    A_prev, W, b = cache
    m = A_prev.shape[1]
    
    
    dW = np.dot(dZ, A_prev.T)/m
    db = np.sum(dZ, axis = 1, keepdims = True)/m
    dA_prev = np.dot(W.T,dZ)
    
    
    assert(dA_prev.shape == A_prev.shape)
    assert(dW.shape == W.shape)
    assert(db.shape == b.shape)
    
    return dA_prev, dW, db



In [14]:
"""
#Backward propogation
activation_backward(dA, cache, activation):
Calculate derivatives of activation function & Z for a given layer, i
"""

def activation_backward(dA, cache, activation):
    
    linear_cache, activation_cache = cache #(A, W, b) & (Z)
    
    
    if activation == "relu":# Might be incorrect
        Z = activation_cache
        dZ = np.array(dA, copy = True)
        dZ[Z<=0] = 0
        
        assert(dZ.shape == Z.shape)
        
        dA_prev, dW, db = linear_backward(dZ, linear_cache)

    elif activation == "sigmoid":
        s = sigmoid(activation_cache)
        dZ = dA* s*(1-s)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
        
    elif activation == "tanh":
        dZ = dA*(1-(np.tanh(activation_cache)**2))
        dA_prev, dW, db = linear_backward(dZ, linear_cache)

    
    return dA_prev, dW, db
    

In [15]:
"""
#Backward propogation
combine activation & linear backwardpropogation gradients across all layers
"""

def backward_propogation(AL, Y, caches, hidden_activation):
    grads = {}
    L = len(caches) #Number of layers
    m = AL.shape[1]
    Y = Y.reshape(AL.shape)
    
    #Derivative of loss function in compute_cost()
    dAL = -(np.divide(Y,AL) - np.divide(1-Y, 1-AL))
    
    #Compute sigmoid derivative
    current_cache = caches[L-1]
    grads["dA"+str(L-1)], grads["dW"+str(L)],grads["db"+str(L)] = activation_backward(dAL, current_cache, "sigmoid")
    
    for l in reversed(range(L-1)):
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = activation_backward(grads["dA"+str(l+1)],current_cache, hidden_activation)
        grads["dA"+str(l)] = dA_prev_temp
        grads["dW"+str(l+1)] = dW_temp
        grads["db"+str(l+1)] = db_temp
        
    return grads

In [16]:

"""
#Gradient Descent
def update_parameters()

modify the weights & biases based on backward_propogation
"""

def update_parameters(parameters ,grads, learning_rate):
    
    L = len(parameters)//2
    
    for l in range(L):
        parameters["W" + str(l+1)] -= learning_rate *grads["dW"+str(l+1)]
        parameters["b" + str(l+1)] -= learning_rate*grads["db"+str(l+1)]
    return parameters
        
        

In [17]:
"""
#Neural Network 

Combine all helper functions (forward_propogation, backward_propogation, compute_cost, update_parameters)
To create an L-layer Neural Network
"""

def L_layer_model(X, Y, layer_dims, hidden_activation, learning_rate, num_iterations, print_cost = False):
    costs = [] # Keep track of cost to plot later
    
    parameters = initialize_parameters(layer_dims)
    
    
    for i in range(0, num_iterations):
        #Forward prop
        AL,caches = forward_propogation(X,parameters, hidden_activation)
        
        #Cost function
        cost = compute_cost(AL,Y)
        
        #Backward prop
        grads = backward_propogation(AL,Y,caches, hidden_activation)
        

        parameters = update_parameters(parameters,grads,learning_rate)
        
        if print_cost and i%100 == 0:
            print("Cost after iteration {}: {}".format(i, np.squeeze(cost)))
            costs.append(cost)
            
    plt.plot(np.squeeze(costs))
    plt.ylabel('cost')
    plt.xlabel('iterations(per hundreds)')
    plt.title("Learning rate = "+str(learning_rate))
    plt.show()
    
    return parameters


In [18]:
"""
Grab the actual training set (4,500+ examples) 
"""
pathOne = "chest_xray/train/NORMAL"
pathTwo = "chest_xray/train/PNEUMONIA"

x_train, y_train = load_xy_set(pathOne,pathTwo)

Iteration:  100 :  <DirEntry 'NORMAL2-IM-0893-0001.jpeg'>
Iteration:  200 :  <DirEntry 'NORMAL2-IM-0633-0001.jpeg'>
Iteration:  300 :  <DirEntry 'IM-0205-0001.jpeg'>
Iteration:  400 :  <DirEntry 'NORMAL2-IM-0621-0001.jpeg'>
Iteration:  500 :  <DirEntry 'IM-0590-0001.jpeg'>
Iteration:  600 :  <DirEntry 'IM-0177-0001.jpeg'>
Iteration:  700 :  <DirEntry 'IM-0348-0001.jpeg'>
Iteration:  800 :  <DirEntry 'NORMAL2-IM-0508-0001.jpeg'>
Iteration:  900 :  <DirEntry 'NORMAL2-IM-0458-0001.jpeg'>
Iteration:  1000 :  <DirEntry 'NORMAL2-IM-0908-0001.jpeg'>
Iteration:  1100 :  <DirEntry 'NORMAL2-IM-0582-0001.jpeg'>
Iteration:  1200 :  <DirEntry 'IM-0517-0001.jpeg'>
Iteration:  1300 :  <DirEntry 'IM-0279-0001.jpeg'>
Iteration:  100 :  <DirEntry 'person531_bacteria_2235.jpeg'>
Iteration:  200 :  <DirEntry 'person1141_virus_1890.jpeg'>
Iteration:  300 :  <DirEntry 'person1433_bacteria_3701.jpeg'>
Iteration:  400 :  <DirEntry 'person74_bacteria_362.jpeg'>
Iteration:  500 :  <DirEntry 'person445_virus_912

In [19]:
#Confirm shape of the traning examples
x_train.shape


(12288, 5216)

In [27]:
"""
Train a random set of Neural Networks to see best performance

'relu' can be replaced with 'tanh' or 'sigmoid', though it is not recommended to use sigmoid

I Used a nested for loop to consider different layer_sizes, different neural network sizes 
(the number of layers in a NN), and different learning_rates.  

Best performing hyperparameters:
Iterations = 1000
Single-layer Neural Network
Hidden layer size = 35 or 25 or 30
hyperparemeter = .099 or .016

Best Accuracy on Test set: 83%
Best Accuracy on Validation set: 94%
Best Accuracy on Train set: 95%
"""

#Constant parameters
input_size = x_train.shape[0]
output_size = 1
hidden_activation = "relu"
iterations = 1000

#Creating of list of hyperparameters to try
#Best performing set of hyperparameters
list_of_h_layers = [25, 35, 35, 25, 30, 30]
learning_rate_list = [.099, .016, .099,.016, .099,.016] #Best learning rates received so far
layer_list = []


parameters = [] #Stores the trained parameters

#Commented out nested for loop after determining 
#the best hyperparameters
for x in range (6):#Number Of test cases
    layer_dims = [input_size]
    layer_dims.append(list_of_h_layers[x])
#    for y in range(1): #Number of hidden layers in test case x
#        hidden_layer_size.append(randint(10,40))
#        layer_dims.append(hidden_layer_size[y])
    
    layer_dims.append(output_size)
    #list_of_h_layers.append(hidden_layer_size)#add to the list
    layer_list.append(layer_dims)
#    learning_rate_list.append(randint(1,100)/1000)#Find the best learning rate

print("How many Neural Networks are we training: ", len(learning_rate_list))
print("List of Learning Rates: ", learning_rate_list)
print("List of Layers dimensions", layer_list)


#Test Neural Networks with differing hyperparameters
for x in range(len(layer_list)):
    parameters.append(L_layer_model(x_train, y_train, layer_list[x],hidden_activation, learning_rate_list[x],iterations, True))

How many Neural Networks are we training:  6
List of Learning Rates:  [0.099, 0.016, 0.099, 0.016, 0.099, 0.016]
List of Layers dimensions [[12288, 25, 1], [12288, 35, 1], [12288, 35, 1], [12288, 25, 1], [12288, 30, 1], [12288, 30, 1]]
Cost after iteration 0: 0.7564716358283298


KeyboardInterrupt: 

In [21]:
#Function to predict with TRAINED NEURAL NETWORKS
def predict(X, Y, parameters, hidden_activation):
    m = X.shape[1]
    n = len(parameters) //2 #Number of layers in the neural network
    
    results, caches = forward_propogation(X, parameters, hidden_activation)
    results = results//(0.5) #Floor divide .5 & receive 1's & 0's
    
    
    accuracy = np.sum((results==Y)/m)
    
    
   # print("Accuracy: " + str(accuracy))
    
    return results, accuracy
    

In [22]:
#Check predictions on the training set -- Should be very accurate(95%+)
p_train_list = []
accuracy_train_list = []
for x in parameters:
    p_train, accuracy_train = predict(x_train,y_train,x, hidden_activation)
    
    p_train_list.append(p_train)
    accuracy_train_list.append(accuracy_train)
    
print("Accuracy of Neural Networks on train_list: ",accuracy_train_list)

Accuracy of Neural Networks on train_list:  [0.9415260736196319, 0.9478527607361963, 0.8757668711656441, 0.9568634969325153]


In [23]:
#Load validation set to test trained Neural Networks
pathOne = "chest_xray/val/NORMAL"
pathTwo = "chest_xray/val/PNEUMONIA"

x_val, y_val = load_xy_set(pathOne,pathTwo)

In [24]:
#Test Neural Networks on validation set
p_val_list = []
accuracy_val_list = []
hidden_activation = hidden_activation
for x in parameters:
    p_val, accuracy_val = predict(x_val, y_val, x, hidden_activation)
    
    p_val_list.append(p_val)
    accuracy_val_list.append(accuracy_val)

#Print accuracy of Neural Networks on validation set
print("Accuracy of Neural Networks on validation_list: ", accuracy_val_list)

Accuracy of Neural Networks on validation_list:  [0.9375, 0.8125, 0.625, 0.9375]


In [25]:
#Print accuracy of Neural Networks on test set
pathOne = "chest_xray/test/NORMAL"
pathTwo = "chest_xray/test/PNEUMONIA"

x_test, y_test = load_xy_set(pathOne, pathTwo)


Iteration:  100 :  <DirEntry 'IM-0081-0001.jpeg'>
Iteration:  200 :  <DirEntry 'NORMAL2-IM-0221-0001.jpeg'>
Iteration:  100 :  <DirEntry 'person113_bacteria_543.jpeg'>
Iteration:  200 :  <DirEntry 'person141_bacteria_681.jpeg'>
Iteration:  300 :  <DirEntry 'person33_virus_72.jpeg'>


In [26]:
p_test_list = []
accuracy_test_list = []

for x in parameters:
        p_test, accuracy_test = predict(x_test, y_test, x, hidden_activation)
        
        p_test_list.append(p_test)
        
        accuracy_test_list.append(accuracy_test)
        
#Print accuracy of Neural Networks on test set
print("Accuracy of Neural Networks on test_list: ",accuracy_test_list)

Accuracy of Neural Networks on test_list:  [0.7932692307692306, 0.721153846153846, 0.6858974358974359, 0.7516025641025641]
